In [63]:
import torch 
import torch.nn as nn
import torchvision 
from torchvision import models,datasets,transforms
from torch.utils.data import DataLoader

In [64]:
train_path = 'Data/train'
test_path = 'Data/validation'

# Transformations 
train_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(torch.tensor([0.5,0.5,0.5]),torch.tensor([0.5,0.5,0.5]))
  ])


test_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(torch.tensor([0.5,0.5,0.5]),torch.tensor([0.5,0.5,0.5]))
  ])

# Dataset 

train_data = datasets.ImageFolder(root=train_path,transform = train_transform)
test_data = datasets.ImageFolder(root=test_path,transform = test_transform)

In [65]:
train_data

Dataset ImageFolder
    Number of datapoints: 44
    Root location: Data/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
           )

In [66]:
test_data

Dataset ImageFolder
    Number of datapoints: 44
    Root location: Data/validation
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
           )

# VGG

In [57]:
vgg = models.vgg11_bn(num_classes=2,pretrained=False)

In [67]:
train_loader = DataLoader(train_data,shuffle=True,batch_size=32)
test_loader = DataLoader(test_data,shuffle=False,batch_size=32)

vgg.train()
n_epochs = 10

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg.parameters())


for epoch in range(n_epochs):
    for batch ,data in enumerate(train_loader):
        optimizer.zero_grad()
        
        pred = vgg(data[0])
        loss = loss_func(pred,data[1])
        loss.backward() 
        optimizer.step()
        
        print(f'Epoch {epoch}/{n_epochs}  loss : {loss.item()}')

Epoch 0/10  loss : 1.120194911956787
Epoch 0/10  loss : 1.8671547174453735
Epoch 1/10  loss : 3.2493181228637695
Epoch 1/10  loss : 0.9218428134918213
Epoch 2/10  loss : 1.3246116638183594
Epoch 2/10  loss : 0.693638265132904
Epoch 3/10  loss : 1.1003941297531128
Epoch 3/10  loss : 1.2613991498947144
Epoch 4/10  loss : 1.088152527809143
Epoch 4/10  loss : 1.885624885559082
Epoch 5/10  loss : 0.6935417652130127
Epoch 5/10  loss : 0.704929530620575
Epoch 6/10  loss : 2.114464044570923
Epoch 6/10  loss : 0.7788889408111572
Epoch 7/10  loss : 2.1979308128356934
Epoch 7/10  loss : 1.2055258750915527
Epoch 8/10  loss : 1.739746332168579
Epoch 8/10  loss : 1.5544229745864868
Epoch 9/10  loss : 1.0130209922790527
Epoch 9/10  loss : 0.8602206707000732


In [68]:
vgg.eval()
n_sample = 0
n_correct = 0


with torch.no_grad():
    for batch, data in enumerate(test_loader):
        x = data[0]
        y = data[1]
        y_pred = vgg(x)
        n_correct += (torch.argmax(y_pred,axis=1) == y).sum()
        n_sample +=y.shape[0]
        


In [69]:
n_sample

44

In [70]:

accuracy = (n_correct/n_sample)*100
accuracy

tensor(50.)

# Check Whether Layers are frozen or not

In [99]:
# Un frezze all layers
for param in vgg.parameters():
    param.requires_grad = True

In [96]:
for name,param in vgg.named_parameters():
    if param.requires_grad:
        print(name,param.numel())
        break

features.0.weight 1728


In [100]:
# Frezze all layers 
for param in vgg.parameters():
    param.requires_grad = False

In [98]:
for name,param in vgg.named_parameters():
    if param.requires_grad:
        print(name,param.numel()) 
else:
    print('None')

None


In [87]:
# VGG Pretrained 

In [103]:
vgg = models.vgg11_bn(num_classes=2,pretrained=True)

vgg.eval()
n_sample = 0
n_correct = 0


with torch.no_grad():
    for batch, data in enumerate(test_loader):
        x = data[0]
        y = data[1]
        y_pred = vgg(x)
        n_correct += (torch.argmax(y_pred,axis=1) == y).sum()
        n_sample +=y.shape[0]

RuntimeError: Error(s) in loading state_dict for VGG:
	size mismatch for classifier.6.weight: copying a param with shape torch.Size([1000, 4096]) from checkpoint, the shape in current model is torch.Size([2, 4096]).
	size mismatch for classifier.6.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([2]).

In [104]:
accuracy = (n_correct/n_sample)*100
accuracy

tensor(0.)

# Conclusion: 


In [ ]:
# 1. Any , Model we are using is attached to its main class 
   # Example if VGG main class
   # vgg11 -- > base class 
   # -- vgg11 -- pretrained = False
   # --- vgg11 -- pretrained = True 

# If pretrained = False: 
   #  you can use it train it , change number of classes 

# If Pretrained = True
   # Then first model will be downloaded 
   # In transfer change the number of class and then retrain it 
   # then eval 
# else: 
  # error 